## Code implementations

### Table of Contents
* [1. Classification](#chapter1)
    * [1.1. Supervised](#section_1_1)
        * [1.1.1. Land Cover Mapping](#sub_section_1_1_1)
        * [1.1.2. Identify Areas of Forest Loss](#sub_section_1_1_2)
        * [1.1.3. Land Cover Analysis](#sub_section_1_1_3)
        * [1.1.4. SVM Image Classifier with TensorFlow](#sub_section_1_1_4)
    * [1.2 Unsupervised](#section_1_2)
        * [1.2.1 Pattern Identification](#section_1_2_1)
* [2. Segmentation](#chapter2)
    * [2.1. Data preprocess](#section_2_1)
    * [2.2. One-hot encoding](#section_2_2)
    * [2.3. Architecture](#section_2_3)
    * [2.4. Loss Function](#section_2_4)
    * [2.5. Model Compliation](#section_2_5)
    * [2.6. Evaluation](#section_2_6)
    * [2.7. Prediction](#section_2_7)
* [3. Other Implementations](#chapter3)
    * [3.1. Earth Engine](#section_3_1)
        * [3.1.1. Data Collection](#sub_section_3_1_1)
        * [3.1.2. Image Preprocess, Export and Load](#sub_section_3_1_2)
        * [3.1.3. Model Training](#sub_section_3_1_3)

## 1. Classification<a class="anchor" id="chapter1"></a>
This is chapter 1
### Supervised 1.1 <a id="section_1_1"></a>
This is section 1.1
#### Land Cover Mapping 1.1.1 <a id="sub_section_1_1_1"></a>
This is sub section 1.1.1
#### Identify Areas of Forest Loss 1.1.2 <a id="sub_section_1_1_2"></a>
This is sub section 1.1.2
#### Land Cover Analysis 1.1.3 <a id="sub_section_1_1_3"></a>
This is sub section 1.1.3
#### SVM Image Classifier with TensorFlow 1.1.4 <a id="sub_section_1_1_4"></a>
This is sub section 1.1.4

## 2. Segmentation<a id="chapter2"></a>
This is chapter number 2
### Data preprocess 2.1 <a id="section_2_1"></a>
This is section 2.1
### One-hot encoding 2.2 <a id="section_2_2"></a>
This is section 2.2
### Architecture 2.3 <a id="section_2_3"></a>
This is section 2.3
### Loss Function 2.4 <a id="section_2_4"></a>
This is section 2.4
### Model Compliation 2.5 <a id="section_2_5"></a>
This is section 2.5
### Evaluation 2.6 <a id="section_2_6"></a>
This is section 2.6
### Prediction 2.7 <a id="section_2_7"></a>
This is section 2.7

## Other Implementations 3 <a id="chapter3"></a>
This is chapter number 3
### Earth Engine 3.1 <a id="section_3_1"></a>
This is section 3.1
#### Data Collection 3.1.1 <a id="sub_section_3_1_1"></a>
This is sub section 3.1.1
#### Image Preprocess, Export and Load 3.1.2 <a id="sub_section_3_1_2"></a>
This is sub section 3.1.2
#### Model Training 3.1.3 <a id="sub_section_3_1_3"></a>
This is sub section 3.1.3